In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import networkx as nx
import numpy as np
from lib.graph_local_classes import GraphStructure, GraphParams, InnerGraphSimulation

In [91]:

from lib.graph_enumerator import generate_graphs
from lib.node_semantics import Node_Name_Rule, Edge_Semantics_Rule
from lib import config, result_config
from lib.likelihood_calculations_shared_params import Inference
from lib.utils import filename_utility
from lib.misc import cond_to_data
import time
from joblib import Parallel, delayed


In [79]:
graph_iter = generate_graphs(**config.generator_dictionary)
graphs = list(graph_iter)
for graph in graphs:
    Node_Name_Rule.graph_semantics_apply(graph,config.node_semantics)
    Edge_Semantics_Rule.graph_semantics_apply(graph,config.edge_semantics)

inference_obj = Inference()

options = config.options
options['data_sets']=cond_to_data(config.conds[1,:])
options['stigma_sample_size']=2000
options['param_sample_size']=2

scale_free_bounds = (10**(-.1),10**(.1))
# scale_free_bounds = (10**(-.25),10**(.25))
# scale_free_bounds = (10**-1,10**1)
# scale_free_bounds = (10**-3,10**3)

options['scale_free_bounds']=scale_free_bounds


In [80]:
result_graphs, result_posterior, result_loglik, result_dict, result_params = inference_obj.p_graph_given_d(graphs,config.options)


[Parallel(n_jobs=-2)]: Done   3 out of   2 | elapsed:  2.0min remaining:  -40.4s
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:  2.0min finished


In [89]:
edges_q = result_config.edges_of_interest.copy()

posterior = result_posterior
for i,g in enumerate(graphs):
    for edge in edges_q:
        if edge in g.edges():
            edges_q[edge]+=posterior[i]



In [87]:
max(result_posterior)

0.63168612198522534

In [90]:
edges_q

{('A_★', 'B_★'): 0.99999999999997113,
 ('A_★', 'C_★'): 0.031644545567001262,
 ('A_★', 'D_★'): 0.042244825208054317,
 ('B_★', 'C_★'): 0.33666899750642104,
 ('B_★', 'D_★'): 0.99999492264721102,
 ('C_★', 'B_★'): 0.031648623135006729,
 ('C_★', 'D_★'): 0.64228132345026334,
 ('D_★', 'B_★'): 0.63169153478740225,
 ('D_★', 'C_★'): 0.67392817765079072}

In [13]:
small_val = .05

cond1 = [0,small_val,small_val,small_val]
cond2 = [0,1,3,2]
cond3 = [0,3,2,1]
cond4 = [0,1,2,2]

conds = np.array([cond1,cond2,cond3,cond4])

In [66]:
result_loglik

array([-1111.80867129, -1112.03018827, -1112.11856113, -1112.15694366,
       -1111.96006903, -1111.91336065, -1112.30524433, -1112.16701269,
       -1111.77595796, -1111.84427046, -1113.56299567, -1112.71611145,
       -1111.99233718, -1112.19199688, -1114.03723794, -1112.50287692,
       -1112.00152314, -1112.22213642, -1112.85197662, -1113.23304345,
       -1112.46588962, -1112.6362465 , -1112.29510593, -1112.90770402,
       -1112.86904984, -1112.26524134, -1112.44143698, -1112.48416211,
       -1113.39024193, -1112.56973999, -1112.21488812, -1112.11182724,
       -1112.30577237, -1112.26730274, -1111.97752776, -1111.9160035 ,
       -1112.54929559, -1112.22267604, -1111.95323729, -1112.08033221,
       -1112.63993439, -1112.38040367, -1112.14550645, -1112.21531653,
       -1112.44165483, -1111.77741963, -1113.09244443, -1113.21914888,
       -1114.10345156, -1113.35569185, -1113.06651706, -1115.20134122,
       -1112.90074577, -1113.25558814, -1113.76517633, -1114.12080943,
      

In [25]:
graph_iter = generate_graphs(**config.generator_dictionary)
graphs = list(graph_iter)
for graph in graphs:
    Node_Name_Rule.graph_semantics_apply(graph,config.node_semantics)
    Edge_Semantics_Rule.graph_semantics_apply(graph,config.edge_semantics)

max_graph = graphs[0]
num_graphs = len(graphs)
# num_params = config.options["param_sample_size"]
num_params = 2
loglikelihood_by_param = np.zeros(shape = (num_params,num_graphs))
print(loglikelihood_by_param.__class__)
max_graph_params = GraphParams.from_networkx(max_graph) # fix this when you can
# max_graph_params.sample()
# blah = inference_obj.subgraph_loglik(graphs[1],max_graph_params,options=config.options)
param_list = [None]*num_params
for i in range(num_params):
    param_list[i] = max_graph_params.sample()
# blah = Parallel(n_jobs = -1, backend = "multiprocessing")( for i,x in enumerate(range(1,10)))
    loglikelihood_by_param = np.array(Parallel(n_jobs=-1, backend="multiprocessing")(
            delayed(inference_obj.subgraph_loglik)(graph, max_graph_params,
                options=options) for graph in graphs))
    
    

<class 'numpy.ndarray'>


In [30]:
for i in range(num_params):
    param_list[i] = max_graph_params.sample()

In [52]:
def helper_subgraph_loglik(graphs,max_graph_params,options=None):
    return np.array([inference_obj.subgraph_loglik(graph,max_graph_params,options=options) for graph in graphs])
loglikelihood_by_param = Parallel(n_jobs=-1, backend="multiprocessing")(
            delayed(helper_subgraph_loglik)(graphs, max_graph_params.from_dict(params),
                options=options) for params in param_list)


# do not go past here

In [53]:
loglikelihood_by_param[0]

array([-393.75169443, -528.55738916, -250.97527905, -396.55414221,
       -439.57018844, -422.42015627, -309.19746986, -415.98352154,
       -431.14525507, -295.97277197,          -inf, -587.10505485,
       -458.35969121, -552.14042652,          -inf, -567.26405325,
       -468.15921252, -760.5689747 , -417.9503687 ,          -inf,
       -367.98772361, -424.52783128, -562.30487535, -288.65458755,
       -195.9473258 , -376.82565402, -450.19947908, -339.95610719,
       -393.36861065, -432.33068766, -487.64272868, -372.55303264,
       -600.26143928, -502.06686648, -575.57201273, -303.01790866,
       -391.3538637 , -324.96956341, -445.00000508, -325.40178431,
       -414.41945005, -388.10687325, -393.93039521, -287.0980737 ,
       -477.9688525 , -244.31721968,          -inf,          -inf,
                -inf,          -inf,          -inf, -413.66913674,
       -493.47837185, -545.85179278, -638.45267836,          -inf,
       -394.5752516 , -424.90646225, -397.83699435,          -

In [54]:
loglikelihood_by_param.shape

AttributeError: 'list' object has no attribute 'shape'

In [26]:
print(loglikelihood_by_param.__class__)


<class 'numpy.ndarray'>


In [134]:
from math import sqrt

In [174]:
Parallel(n_jobs = -1, backend = "multiprocessing")(delayed(sqrt)(x) for i,x in enumerate(range(1,10)))

[1.0,
 1.4142135623730951,
 1.7320508075688772,
 2.0,
 2.23606797749979,
 2.449489742783178,
 2.6457513110645907,
 2.8284271247461903,
 3.0]

In [170]:
test_gen = [delayed(sqrt)(x,idx = i) for i,x in enumerate(range(1,10))]

In [171]:
test_gen

[(<function math.sqrt>, (1,), {'idx': 0}),
 (<function math.sqrt>, (2,), {'idx': 1}),
 (<function math.sqrt>, (3,), {'idx': 2}),
 (<function math.sqrt>, (4,), {'idx': 3}),
 (<function math.sqrt>, (5,), {'idx': 4}),
 (<function math.sqrt>, (6,), {'idx': 5}),
 (<function math.sqrt>, (7,), {'idx': 6}),
 (<function math.sqrt>, (8,), {'idx': 7}),
 (<function math.sqrt>, (9,), {'idx': 8})]

In [128]:
for i,x in enumerate(range(1,5)):
    print(i,x)

TypeError: 'int' object is not iterable

In [121]:
print(max_graph_params.to_dict())
print(max_graph_params.from_dict(param_list[0]).to_dict())
print(max_graph_params.from_dict(param_list[1]).to_dict())


{'r': array([  7.28887461,  57.01761847,   8.54070358,  12.90071227,
        16.59509896,   1.7601836 ,  49.47634301,   1.81576445,
         0.50490592,  20.34303725,  14.63032422,   1.08552227,
        31.78241691,  36.33860303,  14.41687462]), 'lambda0': [18.62627296023096], 'psi_shape': 1.0, 'r_shape': 1.0, 'scale_free_bounds': (0.01, 100), 'names': [('A_int', 'A_obs'), ('A_int', 'A_★'), ('A_★', 'A_obs'), ('A_★', 'B_★'), ('A_★', 'C_★'), ('A_★', 'D_★'), ('B_★', 'B_obs'), ('B_★', 'C_★'), ('B_★', 'D_★'), ('C_★', 'B_★'), ('C_★', 'C_obs'), ('C_★', 'D_★'), ('D_★', 'B_★'), ('D_★', 'C_★'), ('D_★', 'D_obs')], 'mu': array([  2.86044561e+00,   4.79573854e-01,   7.07358855e-01,
         3.49711220e-01,   2.17602414e-01,   5.25812291e-01,
         2.77978135e-01,   5.33475236e-01,   5.75700583e+01,
         1.62187470e-01,   6.05699152e-01,   5.03472105e+01,
         8.27638020e-02,   1.30686555e-02,   4.70509167e-01]), 'n': 15, 'p': 0.8, 'psi': array([ 20.84942935,  27.34415903,   6.04134231,  

In [26]:
class ContObj:
    
    def __iter__(self):
        return iter(['hi', 'banana','red'])
    
    def __getitem__(self, value):
        
        return np.ones(shape=[value]*value)

In [29]:
cont_obj = ContObj()

In [33]:
cont_obj[4][:, :,3, :]

array([[[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]]])

In [25]:
for x in iter(cont_obj):
    print(x)

TypeError: 'ConfObj' object is not iterable

In [8]:
result_graphs